#Problem-1: Implementation of AVL and Red & Black Trees

Read the rules/concept of Red & Black Tree
*   Store 1000 frequent words from the given file in both Trees
*   Search same 100 random words from both data structures and
count comparisons.
*   Prove the statement, AVL trees offer faster search due to their
strict height balancing whereas Red-Black trees are faster while
insert and delete operations.

#Converting text file to csv


In [ ]:
import random
import csv

 # Convert a text file (one word per line) into a CSV file (one word per row)

def txt_to_csv(txt_file, csv_file):

    with open(txt_file, "r") as file:  #open the text file in read mode
        words = [line.strip() for line in file.readlines()] #generating a list

    with open(csv_file, "w", newline="") as file:#Opens CSV file for writing and prevent blank lines
        writer = csv.writer(file) # creating a CSV writer object
        for w in words:
            writer.writerow([w]) #one row with one cell/word




#Applying Readfunction so that file can read the words and can be used to insert in AVL/RB trees + Generate Random Words


In [ ]:
def readFile(filename):
    # Read a text file where each line contains one word.
    # Returns a list of words (strings) without newline characters.
    with open(filename, 'r') as f:
        new_list = [line.strip() for line in f]
    return new_list


def randomWords(data):
    # Create a list of 100 random words chosen from the given list 'data'.

    random_words = [] #since selection is random so possiblility of picking duplicate words
    for i in range(100):
        key = random.choice(data)
        random_words.append(key)
    return random_words


#Code for defining AVL tree

In [ ]:

class AVLNode:
    # Node class for AVL Tree
    def __init__(self, value):#Value is data stored in node, for our case it is word
        self.value = value
        self.left = None #pointer to the left child
        self.right = None #pointer to the right child
        self.height = 1 # a new node is leaf so height is 1


class AVLTree:
    # AVL Tree class
    def __init__(self): #self.root stores the top node of tree
        self.root = None #when tree is empty, root =None

    def height(self, node):
        # Return height of a node
        if not node: #if node is none, height=0
            return 0
        return node.height

    def balance(self, node): # Return balance factor = height(left) - height(right)
        if not node:
            return 0
        return self.height(node.left) - self.height(node.right) #Between <-1 (leaning right) and +1(leaning left),  tree is balanced at this node
#AVL trees try to keep this in [-1, 0, 1] for every node.

    def right_rotate(self, node): #Rotations- fixing when it leans too much by re arranging the nodes
        # Right rotation around given node
        new_root = node.left
        temp = new_root.right

        new_root.right = node
        node.left = temp

        # Update heights
        node.height = 1 + max(self.height(node.left), self.height(node.right))
        new_root.height = 1 + max(self.height(new_root.left), self.height(new_root.right))
        return new_root

    def left_rotate(self, node):
        # Left rotation around given node
        new_root = node.right
        temp = new_root.left

        new_root.left = node
        node.right = temp

        # Update heights
        node.height = 1 + max(self.height(node.left), self.height(node.right))
        new_root.height = 1 + max(self.height(new_root.left), self.height(new_root.right))
        return new_root

    def insert(self, root, value):
        # Insert a value into the AVL tree and keep it balanced
        if not root:
            return AVLNode(value) #if root is non, place new node here.
        elif value < root.value: #go to left subtree.
            root.left = self.insert(root.left, value)
        else:
            # Note: duplicates will go to the right in this version
            root.right = self.insert(root.right, value) #go to right subtree.

        # Update height
        root.height = 1 + max(self.height(root.left), self.height(root.right))
        balance = self.balance(root)

        # Left-left case
        if balance > 1 and value < root.left.value: #if heavy on left-left side, do a right rotation
            return self.right_rotate(root)

        # Right-right case
        if balance < -1 and value > root.right.value: #heavy on right-right side, Do one left rotation.
            return self.left_rotate(root)

        # Left-right case
        if balance > 1 and value > root.left.value:#heavy on left-right side, First left rotate the left child, then right rotate the node.
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        # Right-left case
        if balance < -1 and value < root.right.value:#heavy on right-left side , First right rotate the right child, then left rotate the node.
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root #Return the (possibly rotated) root of this subtree.
        #Searching with comparison counting

# NEW: search that also counts comparisons
    #Start at root.
    #Each time you compare value with current.value, you increment comparisons.
    #If equal → found → return the node and the count.
    #If smaller → go left.
    #If greater → go right.
    #If you fall off the tree (current becomes None) → not found, but you still return how many comparisons were done.

      # Standard BST search on the AVL tree that counts comparisons.
    # ===========================
    # DELETE OPERATION (ADD THIS)
    # ===========================

    def delete(self, root, value):
        # 1. Standard BST deletion
        if not root:
            return root
        elif value < root.value:
            root.left = self.delete(root.left, value)
        elif value > root.value:
            root.right = self.delete(root.right, value)
        else:
            # Node found
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left

            # Replace with min value in right subtree
            min_larger_node = self._min_value_node(root.right)
            root.value = min_larger_node.value
            root.right = self.delete(root.right, min_larger_node.value)

        # 2. Update height
        root.height = 1 + max(self.height(root.left), self.height(root.right))

        # 3. Rebalance
        balance = self.balance(root)

        # Left heavy
        if balance > 1:
            if self.balance(root.left) >= 0:
                return self.right_rotate(root)
            else:
                root.left = self.left_rotate(root.left)
                return self.right_rotate(root)

        # Right heavy
        if balance < -1:
            if self.balance(root.right) <= 0:
                return self.left_rotate(root)
            else:
                root.right = self.right_rotate(root.right)
                return self.left_rotate(root)

        return root

    def _min_value_node(self, node):
        while node.left:
            node = node.left
        return node


    def search_with_count(self, root, value):
        current = root
        comparisons = 0

        while current is not None:
            comparisons += 1
            if value == current.value:
                return current, comparisons
            elif value < current.value:
                current = current.left
            else:
                current = current.right

        return None, comparisons


#Code for defining red tree

In [ ]:



class Node:
    # Node class for Red-Black Tree
    def __init__(self, value, color='red'):
        self.value = value
        self.color = color      # 'red' or 'black'
        self.left = None
        self.right = None
        self.parent = None

    def grandparent(self):
        # Return grandparent of this node
        if self.parent is None:
            return None
        return self.parent.parent

    def sibling(self):
        # Return sibling of this node
        if self.parent is None:
            return None
        if self == self.parent.left:
            return self.parent.right
        return self.parent.left

    def uncle(self):
        # Return uncle of this node
        if self.parent is None:
            return None
        return self.parent.sibling()


class RedBlackTree:
    # Red-Black Tree class
    def __init__(self):
        self.root = None

    def search(self, value):
        # Normal BST search (without counting)
        curr_node = self.root
        while curr_node is not None:
            if value == curr_node.value:
                return curr_node
            elif value < curr_node.value:
                curr_node = curr_node.left
            else:
                curr_node = curr_node.right
        return None

    # NEW: search that counts comparisons
    def search_with_count(self, value):
        curr_node = self.root
        comparisons = 0

        while curr_node is not None:
            comparisons += 1  # comparing value with curr_node.value
            if value == curr_node.value:
                return curr_node, comparisons
            elif value < curr_node.value:
                curr_node = curr_node.left
            else:
                curr_node = curr_node.right

        return None, comparisons

    def insert(self, value):
        # Regular BST-style insert first
        new_node = Node(value)  # new nodes are red by default
        if self.root is None:
            self.root = new_node
        else:
            curr_node = self.root
            while True:
                if value < curr_node.value:
                    if curr_node.left is None:
                        curr_node.left = new_node
                        new_node.parent = curr_node
                        break
                    else:
                        curr_node = curr_node.left
                else:
                    # Handle duplicate: ignore it
                    if value == curr_node.value:
                        return
                    else:
                        if curr_node.right is None:
                            curr_node.right = new_node
                            new_node.parent = curr_node
                            break
                        else:
                            curr_node = curr_node.right

        # Fix colors and rotations to maintain RB properties
        self.insert_fix(new_node)

    def insert_fix(self, new_node):
        # Restore Red-Black properties after insertion
        while new_node.parent and new_node.parent.color == 'red':
            if new_node.parent == new_node.grandparent().left:
                uncle = new_node.uncle()
                if uncle and uncle.color == 'red':
                    new_node.parent.color = 'black'
                    uncle.color = 'black'
                    new_node.grandparent().color = 'red'
                    new_node = new_node.grandparent()
                else:
                    if new_node == new_node.parent.right:
                        new_node = new_node.parent
                        self.rotate_left(new_node)
                    new_node.parent.color = 'black'
                    new_node.grandparent().color = 'red'
                    self.rotate_right(new_node.grandparent())
            else:
                uncle = new_node.uncle()
                if uncle and uncle.color == 'red':
                    new_node.parent.color = 'black'
                    uncle.color = 'black'
                    new_node.grandparent().color = 'red'
                    new_node = new_node.grandparent()
                else:
                    if new_node == new_node.parent.left:
                        new_node = new_node.parent
                        self.rotate_right(new_node)
                    new_node.parent.color = 'black'
                    new_node.grandparent().color = 'red'
                    self.rotate_left(new_node.grandparent())

        self.root.color = 'black'



    def rotate_left(self, node):
        right_child = node.right
        node.right = right_child.left

        if right_child.left is not None:
            right_child.left.parent = node

        right_child.parent = node.parent

        if node.parent is None:
            self.root = right_child
        elif node == node.parent.left:
            node.parent.left = right_child
        else:
            node.parent.right = right_child

        right_child.left = node
        node.parent = right_child

    def rotate_right(self, node):
        left_child = node.left
        node.left = left_child.right

        if left_child.right is not None:
            left_child.right.parent = node

        left_child.parent = node.parent

        if node.parent is None:
            self.root = left_child
        elif node == node.parent.right:
            node.parent.right = left_child
        else:
            node.parent.left = left_child

        left_child.right = node
        node.parent = left_child

    def _replace_node(self, old_node, new_node):
        # Helper for replace (not used in this assignment, but kept from your code)
        if old_node.parent is None:
            self.root = new_node
        else:
            if old_node == old_node.parent.left:
                old_node.parent.left = new_node
            else:
                old_node.parent.right = new_node
        if new_node is not None:
            new_node.parent = old_node.parent

    def _find_min(self, node):
        # Find minimum value node in a subtree
        while node.left is not None:
            node = node.left
        return node

    def _inorder_traversal(self, node):
        # Inorder traversal (for debugging/printing, not required for assignment)
        if node is not None:
            self._inorder_traversal(node.left)
            print(node.value, end=" ")
            self._inorder_traversal(node.right)





#Main Function Execution

In [ ]:
def main():


    #   Read 1000 words from the TXT file ----

    words = readFile('1000 Frequent Words.txt')
    #print(f"Read {len(words)} words from '{txt_file}'.")

    #  Create both trees and insert all 1000 words ----
    rb_tree = RedBlackTree()
    avl_tree = AVLTree()

    for w in words:
        rb_tree.insert(w)                # insert into Red-Black Tree
        avl_tree.root = avl_tree.insert(avl_tree.root, w)  # insert into AVL Tree

    print("Inserted all words into both trees successfully!")

    # Generate 100 random words from the same list ----
    random_100 = randomWords(words)
    print("Generated 100 random words for searching.")

    #   Search those 100 words in both trees and count comparisons ----
    total_rb_comparisons = 0
    total_avl_comparisons = 0

    for word in random_100:
        # AVL search
        _, avl_comps = avl_tree.search_with_count(avl_tree.root, word)
        total_avl_comparisons += avl_comps

        # Red-Black search
        _, rb_comps = rb_tree.search_with_count(word)
        total_rb_comparisons += rb_comps



    #   Print comparison results
    print("\n===== Search Comparison Results (100 random words) =====")
    print(f"Total comparisons in AVL Tree:       {total_avl_comparisons}")
    print(f"Average comparisons per search AVL:  {total_avl_comparisons / 100:.2f}")
    print(f"Total comparisons in Red-Black Tree: {total_rb_comparisons}")
    print(f"Average comparisons per search RB:   {total_rb_comparisons / 100:.2f}")


if __name__ == "__main__":
    main()

Inserted all words into both trees successfully!
Generated 100 random words for searching.


AttributeError: 'NoneType' object has no attribute 'right'